# Force Calculator

Given a set of parameters, this notebook calculates the required amperage in the wire to flip the magnet.

The following contains a diagram of the setup.

![setup diagram](diagrams/setup.png)

## Imports

In [4]:
import sympy as sym
from sympy.vector import CoordSys3D
from scipy import constants

## Parameters

In [5]:
### Parameters

# Solenoid Parameters
wire_radius = 0.001 # m
number_of_turns = 100
turns_per_level = 1 # how many coils are at the same level

# Dipole Position Parameters
dipole_z = 0.1 # m
dipole_x = 0 # m, in the direction of r_hat where x = 0 is the center of the solenoid

# Dipole Properties
dipole_moment_magnitude = 0.0168 # A m^2. Use https://www.kjmagnetics.com/calculator.asp to calculate this.

## Magnetic Field Equation

In [29]:
C = CoordSys3D('C', transformation='cylindrical', vector_names=list('rtk'), variable_names=list('RTK'))

(
D, # Radius of the solenoid
r_0, # Distance along theta = 0 from the center of the solenoid
z_0, # Distance along z from the center of the solenoid
theta, # Angle from the center of the solenoid
pi, # Pi
mu_0, # Permeability of free space
) = sym.symbols('D r_0 z_0 theta pi mu_0', real=True, positive=True)

I = sym.symbols('I', real=True) # Current in the solenoid

# Magnetic field of a solenoid
B_original = mu_0*I*D / (4 * pi) * sym.integrate((z_0 * C.r + (D - r_0) * C.k) / (D**2 - 2*r_0*D*sym.cos(theta) + r_0**2 + z_0**2)**(3/2), (theta, 0, 2*pi))
B_point = B_original.subs({ mu_0: constants.mu_0, pi: constants.pi })

Testing...

In [31]:
# Compare to the answer 6.2831 * 10^-7 k_hat
B_point.subs({ r_0: 0, z_0: 0, D: 1, I: 1 }).simplify()

(6.2831853106e-7)*C.k